# Recurrent Neural Networks

Import Data

In [21]:
import numpy as np

# data I/O
data = open('input.txt', 'r').read() # should be simple plain text file
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
#print(list(set(data)))
#print(data_size)
#print(vocab_size)
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }
#print(char_to_ix)
#print(ix_to_char)

In [22]:
hidden_size = 100 # size of hidden layer of neurons
seq_length = 25 # number of steps to unroll the RNN for
learning_rate = 1e-1

In [23]:
Wxh = np.random.randn(hidden_size, vocab_size)*0.01 # input to hidden
Whh = np.random.randn(hidden_size, hidden_size)*0.01 # hidden to hidden
Why = np.random.randn(vocab_size, hidden_size)*0.01 # hidden to output
bh = np.zeros((hidden_size, 1)) # hidden bias
by = np.zeros((vocab_size, 1)) # output bias

In [33]:
def lossFun(inputs, targets, hprev):
    """
    inputs,targets are both list of integers.
    hprev is Hx1 array of initial hidden state
    returns the loss, gradients on model parameters, and last hidden state
    """
    xs, hs, ys, ps = {}, {}, {}, {}
    hs[-1] = np.copy(hprev)
    loss = 0
    
    # forward pass
    for t in range(len(inputs)):
        xs[t] = np.zeros((vocab_size,1)) # encode in 1-of-k representation
        xs[t][inputs[t]] = 1
        hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh) # hidden state
        ys[t] = np.dot(Why, hs[t]) + by # unnormalized log probabilities for next chars
        ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars
        loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss)
    
    # backward pass: compute gradients going backwards
    dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
    dbh, dby = np.zeros_like(bh), np.zeros_like(by)
    dhnext = np.zeros_like(hs[0])
    for t in reversed(range(len(inputs))):
        dy = np.copy(ps[t])
        dy[targets[t]] -= 1 # backprop into y. see http://cs231n.github.io/neural-networks-case-study/#grad if confused here

        dWhy += np.dot(dy, hs[t].T) #ok
        dby += dy                   #ok

        dh = np.dot(Why.T, dy) + dhnext # backprop into h
        dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity

        dbh += dhraw

        dWxh += np.dot(dhraw, xs[t].T)
        dWhh += np.dot(dhraw, hs[t-1].T)

        dhnext = np.dot(Whh.T, dhraw)

    for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
        np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients
    return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

In [29]:
def sample(h, seed_ix, n):
    """
    sample a sequence of integers from the model
    h is memory state, seed_ix is seed letter for first time step
    """
    x = np.zeros((vocab_size, 1))
    x[seed_ix] = 1
    ixes = []
    for t in range(n):
        h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
        y = np.dot(Why, h) + by
        p = np.exp(y) / np.sum(np.exp(y))
        #ix = np.random.choice(range(vocab_size), p=p.ravel())
        ix = np.argmax(p)
        x = np.zeros((vocab_size, 1))
        x[ix] = 1
        ixes.append(ix)
    return ixes

In [34]:
n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variables for Adagrad
smooth_loss = -np.log(1.0/vocab_size)*seq_length # loss at iteration 0
while True:
    # prepare inputs (we're sweeping from left to right in steps seq_length long)
    if p+seq_length+1 >= len(data) or n == 0:
        hprev = np.zeros((hidden_size,1)) # reset RNN memory
        p = 0 # go from start of data
    inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
    targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]
                
    # sample from the model now and then
    if n % 100 == 0:
        sample_ix = sample(hprev, inputs[0], 200)
        txt = ''.join(ix_to_char[ix] for ix in sample_ix)
        print('----\n %s \n----' % (txt, ))
        
    # forward seq_length characters through the net and fetch gradient
    loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
    smooth_loss = smooth_loss * 0.999 + loss * 0.001
    if n % 100 == 0: print('iter %d, loss: %f' % (n, smooth_loss)) # print progress
                    
    # perform parameter update with Adagrad
    for param, dparam, mem in zip([Wxh, Whh, Why, bh, by],[dWxh, dWhh, dWhy, dbh, dby],[mWxh, mWhh, mWhy, mbh, mby]):
        mem += dparam * dparam
        param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update
                                
    p += seq_length # move data pointer
    n += 1 # iteration counter

----
 zvjp9vjp9vjp9vjp9vjp9vjp9vjp9vjp9vjp9vjp9vjp9vjp9vjp9vjp9vjp9vjp9vjp9vjp9vjp9vjp9vjp9vjp9vjp9vjp9vjp9vjp9vjp9vjp9vjp9vjp9vjp9vjp9vjp9vjp9vjp9vjp9vjp9vjp9vjp9vjp9vjp9vjp9vjp9vjp9vjp9vjp9vjp9vjp9vjp9vjp 
----
iter 0, loss: 105.487697
----
  t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t t 
----
iter 100, loss: 106.900735
----
 enenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenenen 
----
iter 200, loss: 104.911511
----
 d d d d d d d d d d d d d d d d d d d d d d d d d d d d d d d d d d d d d d d d d d d d d d d d d d d d d d d d d d d d d d d d d d d d d d d d d d d d d d d d d d d d d d d d d d d d d d d d d d d d  
----
iter 300, loss: 102.697680
----
  den den den den den den den den den

----
  Sponder Schas dastenen den den den den den den den den den den den den den den den den den den den den den den den den den den den den den den den den den den den den den den den den den den den den  
----
iter 3500, loss: 52.823760
----
 er Vaber Vaber den das der Vaber Vaber den das der Vaber Vaber den das der Vaber Vaber den das der Vaber Vaber den das der Vaber Vaber den das der Vaber Vaber den das der Vaber Vaber den das der Vaber 
----
iter 3600, loss: 52.185878
----
 das Se Gen (Ostie dass ind die vor dei Sorren dass ind die vor dei Sorren dass ind die vor dei Sorren dass ind die vor dei Sorren dass ind die vor dei Sorren dass ind die vor dei Sorren dass ind die v 
----
iter 3700, loss: 51.511512
----
 ter aung das den Schender Vaginalenon dei Vaginalenon dei Vaginalenon dei Vaginalenon dei Vaginalenon dei Vaginalenon dei Vaginalenon dei Vaginalenon dei Vaginalenon dei Vaginalenon dei Vaginalenon de 
----
iter 3800, loss: 50.865226
----
  Vaginalen (inder Vagine füren Sch

----
 ei das den Vaginae Scheiten vorder agen lich abelen und an Ender (Fonond mit der Vaginae Scheiten vorder agen lich abelen und an Ender (Fonond mit der Vaginae Scheiten vorder agen lich abelen und an E 
----
iter 7000, loss: 35.399854
----
 er Vuture FoUnhen der Scheiden vorder in der Vuturter Scheiden vorder in der Vuturter Scheiden vorder in der Vuturter Scheiden vorder in der Vuturter Scheiden vorder in der Vuturter Scheiden vorder in 
----
iter 7100, loss: 35.092982
----
 den der (Scheiden. Die nochsal (bixco dir agenon (der Vagen. Die nast abellen Dale man Scheidengewölbe (Formandans aund der (Scheiden. Die nochsal (bixco dir agenon (der Vagen. Die nast abellen Dale m 
----
iter 7200, loss: 34.837647
----
 s dein) wirter Vutgerallen siche dan Schenderm und den (Scheid und den Schenden, tiefönchtit die Vorderen (Porhevagina der Vaginalie der Gebärmutterhanden (Scheid und den Schenden, tiefönchtit die Vor 
----
iter 7300, loss: 34.504610
----
 rter Alge sie der dal aunkön aund 

----
 in den Masster Scheiden. Die nicheichen dem Mabelle (Frtale der Gebärmutterhanden, die Scheiden vorderalstals allen Dast und vertit der Vuteren, die Scheiden vaginalie vagen (Deisalie Dast und vertit  
----
iter 10500, loss: 26.977705
----
 en, die Mast abe dienen. Gebärmutter vorderal stin sablal den Scheid und dass der Scheiden. Die Scheidengewölbe (Fornale mit der Tabelle Gebärmutter vorderal stit der Gebärmutterse wichter (Cormirch a 
----
iter 10600, loss: 26.793533
----
 ochmal der Haschen dass Bensekönich auch vaschen daschen schenan, eit ber Kond forr avornh monderhanden den Magfllchtele Spabel den kanfene (vormals eingen (Schenden Scheiter in der Vagen) vors der Ge 
----
iter 10700, loss: 26.547231
----
 ien dast der Vagina mürsit dam und der Spas den kagen und der Baunden) den Mastdar Senter Tabelle müsste dann din dem Kourter Alleinae Scheidengen, kabllen abgen) mit der Harnd mit das dinter Tabelle  
----
iter 10800, loss: 26.319558
----
 it den kannde den Tabellen und

----
 lin den zwister uchen der zwischen dem Mast die der zwischen dem Mast die der zwischen dem Mast die der zwischen dem Mast die der zwischen dem Mast die der zwischen dem Mast die der zwischen dem Mast  
----
iter 13900, loss: 21.199026
----
 d das Septum veief (ind urter avoralsen und das Septum verinagen vorte seitarsalit der Hass das Selle (Scheid und dass in der Tabelle müsstin disch auche worseidarsale müsstin disch auche worseidarsal 
----
iter 14000, loss: 21.090363
----
  der Harn damit re nden Konlegen umat den „eingereite wichte dass den FUX-Teil der Vaginae oles aurcheichen sereitersas ing Dale Hannd der Schee Spraginge Scheiden kanf hie damit das ind und muttersal 
----
iter 14100, loss: 20.912412
----
 on und der Baungenen. Die SPO-32 darsanin der Spatt rin der Hasn dast sichme münd der das den das Bei der nochmaum, s-32 darsaninae ovorhrats sichten sie Vagina mürsit dam Tabelle mit nicheichen henal 
----
iter 14200, loss: 20.672138
----
 ch ie Übermals ief (dst. Die V

...

----
 Bemerkungen).
Am Ende der Tabelle nicht. Michael Glunk: Ich weiß nicht genau, welche Lehrveranstaltungen (inkl. LB) wir hierfür noch in die Übersicht für den Master G aufnehmen müssen. Das können wir  
----
iter 145300, loss: 0.509543
----
 ass wir immer nur den zusätzlichen deputatsrelevanten Umfang in den Tabellen haben. Sonst passt die Rechnung am Ende der Tabelle müsste dann das durchschnittliche Deputat für jeden Kollegen ersichtlic 
----
iter 145400, loss: 0.506092
----
 isch abklären. Vielen Dank und viele Grüße und ein schönes Wochenende. Die Vagina mündet mit der Scheidenöffnung (Ostium vaginae oder Introitus vaginae) in den Scheidenvorhof (Vestibulum vaginae) der  
----
iter 145500, loss: 0.502563
----
 net wird. Ist die Scheidenöffnung normal ausgeprägt, kann das Menstruationsblut abfließen. Gebärmutterseitig ragt die Portio vaginalis uteri des Gebärmutterhalses (Cervix uteri) in die Vagina vor. Die 
----
iter 145600, loss: 0.499072
----
 vorderen (ventralen) flachen S

----
 32 darstellen. Conny Richter: Kannst Du bitte noch den FUX-Teil durchschauen und alle Zeilen löschen, die bereits schon bei F,FR aufgeführt sind, so dass wir immer nur den zusätzlichen deputatsrelevan 
----
iter 148700, loss: 0.467474
----
 rsolel dietwand (Dorsalwand) der Vagina ist durch Bindegewebe (Septum rectovaginale) mit dem Mastdarm und die Vorderwand über das Septum vesicovaginale und das Septum urethrovaginale mit der Harnblase 
----
iter 148800, loss: 0.742190
----
 Schegen (in Jurtderepr. Das können wir ggf. am Montag nochmal tie ber aungfanster aunker: Vaginae) Ist um gentlich auch ier ind genau, welest, ben daschen, die bereits schon bei F,FR aufgeführt sind,  
----
iter 148900, loss: 1.199701
----
 i) in die Vagina vor. Diese wird von der Vagina so umfasst, dass sie zwischen dem tiefen, hinteren (dorsalen), sowie dem Michtalindene Amntra) riche fungtfon dast die Vagina mündet mit der Spalte vagi 
----
iter 149000, loss: 1.456827
----
 rmige Aussackung des Bauchfell

----
 ätzlichen deputatsrelevanten Umfang in den Tabellen haben. Sonst passt die Rechnung am Ende der Tabelle nicht. Michael Glunk: Ich weiß nicht genau, welche Lehrveranstaltungen (inkl. LB) wir hierfür no 
----
iter 152100, loss: 0.603692
----
 k und viele Grüße und ein schönes Wochenende. Die Vagina mündet mit der Scheidenöffnung (Ostium vaginae oder Introitus vaginae) in den Scheidenvorhof (Vestibulum vaginae) der Vulva. Bei manchen Frauen 
----
iter 152200, loss: 0.588501
----
 nöffnung normal ausgeprägt, kann das Menstruationsblut abfließen. Gebärmutterseitig ragt die Portio vaginalis uteri des Gebärmutterhalses (Cervix uteri) in die Vagina vor. Diese wird von der Vagina so 
----
iter 152300, loss: 0.575275
----
 hen Scheidengewölbe (Fornix vaginae) liegt. Das hintere Scheidengewölbe reicht bis an den Douglas-Raum (Excavatio rectouterina) heran, einer taschenförmige Aussackung des Bauchfells (Peritoneum) und d 
----
iter 152400, loss: 0.563433
----
 egewebe (Septum rectovaginale)

----
 (inkl. LB) wir hierfür noch in die Übersicht für den Master G aufnehmen müssen. Das können wir ggf. am Montag nochmal telefonisch abklären. Vielen Dank und viele Grüße und ein schönes Wochenende. Die  
----
iter 155500, loss: 1.988584
----
 Vulva. Bei manchen Frauen und die Vorderwand über das Septum vesicovaginale und das Septum urethrovaginae) in den Scheidenvorhof (Vestibulum vaginae) der Vulva. Bei manchen Frauen und damit der tiefst 
----
iter 155600, loss: 1.878647
----
 se wird von der Vagina so umfasst, dass sie zwischen dem tiefen, hinteren (dorsalen), sowie dem kleineren seitlichen und dem vorderen (ventralen) flachen Scheidengewölbe (Fornix vaginae) liegt. Das hi 
----
iter 155700, loss: 1.784129
----
 hfells (Peritoneum) und damit der tiefste Teil des Bauchraums zwischen Mastdarm und Gebärmutter. Die Hinterwand (Dorsalwand) der Vagina ist durch Bindegewebe (Septum rectovaginale) mit dem Mastdarm un 
----
iter 155800, loss: 1.678168
----
  Der Gebärmutterhals ist um et

KeyboardInterrupt: 

In [ ]:
dy = np.copy(ps[t])
dy[targets[t]] -= 1 # backprop into y. see http://cs231n.github.io/neural-networks-case-study/#grad if confused here

dh = np.dot(Why.T, dy) + dhnext # backprop into h
dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity


dWhh += np.dot(dhraw, hs[t-1].T)
dhnext = np.dot(Whh.T, dhraw)